In [475]:
import numpy as np
from sklearn.base import BaseEstimator, ClassifierMixin
import arff
import numpy.linalg as la
from sklearn.model_selection import train_test_split

In [571]:
np.set_printoptions(precision=6)
np.set_printoptions(suppress=True)

In [497]:
def normalize(X, min_val=None, max_val=None):
    if min_val is None: min_val = np.nanmin(X, axis=0)
    if max_val is None: max_val = np.nanmax(X, axis=0)
    return (X - min_val) / (max_val - min_val)

class KNNClassifier(BaseEstimator,ClassifierMixin):
    def __init__(self, k, output_type="nominal", input_type=0, weight_type='inverse_distance', normalize=True):
        """
        Args:
            columntype for each column tells you if continues[real] or if nominal.
            weight_type: inverse_distance voting or if non distance weighting. Options = ["no_weight","inverse_distance"]
        """
        self.output_type = output_type
        self.input_type = input_type
        if type(input_type) == np.ndarray:
            self.num_noms = sum(input_type)
        self.weight_type = weight_type
        self.normalize = normalize
        self.k = k
        
    def fit(self, X, y):
        """ Fit the data; run the algorithm (for this lab really just saves the data :D)
        Args:
            X (array-like): A 2D numpy array with the training data, excluding targets
            y (array-like): A 2D numpy array with the training targets
        Returns:
            self: this allows this to be chained, e.g. model.fit(X,y).predict(X_test)
        """
        if type(self.input_type) == np.ndarray:
            self.X_nom = X.T[self.input_type==1].T
            X_cont = X.T[self.input_type==0].T
            if self.normalize:
                self.min_val = np.nanmin(X_cont, axis=0)
                self.max_val = np.nanmax(X_cont, axis=0)
                self.X_cont = normalize(X_cont)
        else:
            if self.normalize:
                self.min_val = np.nanmin(X, axis=0)
                self.max_val = np.nanmax(X, axis=0)
                self.X = normalize(X)
            else:
                self.X = X
        
        if self.output_type == "nominal":
            self.y = y.flatten().astype(int)
            self.num_classes = np.max(self.y) + 1
        else:
            self.y = y.flatten()
        return self
    
    def predict(self, X):
        if type(self.input_type) == np.ndarray and self.normalize:
            X_nom = X.T[self.input_type==1].T
            X_cont = X.T[self.input_type==0].T
            X_cont = normalize(X_cont, min_val=self.min_val, max_val=self.max_val)
            X = np.hstack([X_nom, X_cont])
        elif self.normalize:
            X = normalize(X, min_val=self.min_val, max_val=self.max_val)
        
        if self.output_type == "nominal":
            y_hat = self.predict_nominal(X)
        else:
            y_hat = self.predict_regress(X)
        return y_hat
    
    def topk(self, x):
        if type(self.input_type) == np.ndarray:
            x_nom = x[:self.num_noms]
            x_cont = x[self.num_noms:]
            nom_diff = np.sum(self.X_nom != x_nom, axis=1)
            cont_diff = self.X_cont - x_cont
            cont_diff[np.isnan(cont_diff)] = 1
            norms = np.sqrt(np.sum(cont_diff**2, axis=1) + nom_diff)
        else:
            norms = la.norm(self.X - x, axis=1)
        Ny = np.vstack([norms, self.y]).T
        sort_arg = np.argsort(norms)
        topk = Ny[sort_arg][:self.k]
        return topk
    
    def predict_nominal(self, X):
        """ Predict all classes for a dataset X
        Args:
            X (array-like): A 2D numpy array with the training data, excluding targets
        Returns:
            array, shape (n_samples,)
                Predicted target values per element in X.
        """
        y_hat = []
        for x in X:
            topk = self.topk(x)
            if self.weight_type == "no_weight":
                (values,counts) = np.unique(topk[:, -1].astype(int),return_counts=True)
                ind = np.argmax(counts)
                y_hat.append(values[ind])
            else:
                weights = topk[:, 0]**(-2)
                best_weight = 0
                best_ind = 0
                for i in range(self.num_classes):
                    curr_weight = np.sum(weights[topk[:, -1] == i])
                    if curr_weight > best_weight:
                        best_weight = curr_weight
                        best_ind = i
                y_hat.append(best_ind)
        return np.array(y_hat).reshape(-1, 1)
    
    def predict_regress(self, X):
        y_hat = []
        for x in X:
            topk = self.topk(x)
            if self.weight_type == "no_weight":
                y_hat.append(np.mean(topk[:, -1]))
            else:
                weights = topk[:, 0]**(-2)
                val = np.sum(weights*topk[:, -1])/np.sum(weights)
                y_hat.append(val)
        return np.array(y_hat).reshape(-1, 1)

    #Returns the Mean score given input data and labels
    def score(self, X, y):
        """ Return accuracy of model on a given dataset. Must implement own score function.
        Args:
                X (array-like): A 2D numpy array with data, excluding targets
                y (array-like): A 2D numpy array with targets
        Returns:
                score : float
                        Mean accuracy of self.predict(X) wrt. y.
        """
        y_hat = self.predict(X)
        if self.output_type == "nominal":
            return sum(y_hat==y) / len(y)
        else:
            return np.sum((y_hat - y)**2) / len(y)

# Debug

In [463]:
seismic_train = arff.Arff(arff=r'seismic-bumps_train.arff', label_count=1)
seismic_test = arff.Arff(arff=r'seismic-bumps_test.arff', label_count=1)
X_train = seismic_train[:, :-1]
y_train = seismic_train[:, -1].reshape(-1, 1)
X_test = seismic_test[:, :-1]
y_test = seismic_test[:, -1].reshape(-1, 1)
#knn = KNNClassifier(3, weight_type="no_weight")
knn = KNNClassifier(3, normalize=False)
knn.fit(X_train, y_train)
y_hat = knn.predict(X_test)
score = knn.score(X_test, y_test)
score

array([0.93571429])

# Evaluation

In [131]:
diabetes_train = arff.Arff(arff=r'diabetes.arff', label_count=1)
diabetes_test = arff.Arff(arff=r'diabetes_test.arff', label_count=1)
X_train = diabetes_train[:, :-1]
y_train = diabetes_train[:, -1].reshape(-1, 1)
X_test = diabetes_test[:, :-1]
y_test = diabetes_test[:, -1].reshape(-1, 1)
#knn = KNNClassifier(3, weight_type="no_weight")
knn = KNNClassifier(3)
knn.fit(X_train, y_train)
y_hat = knn.predict_nominal(X_test)

In [133]:
sum(y_test == y_hat)/len(y_test)

array([0.86588542])

In [136]:
np.savetxt("diabetes_prediction.csv", y_hat.astype(int), delimiter=',')

# Telescope

## Raw

In [466]:
telescope_train = arff.Arff(arff=r'telescope_train.arff', label_count=1)
telescope_test = arff.Arff(arff=r'telescope_test.arff', label_count=1)
X_train = telescope_train[:, :-1]
y_train = telescope_train[:, -1].reshape(-1, 1)
X_test = telescope_test[:, :-1]
y_test = telescope_test[:, -1].reshape(-1, 1)
knn = KNNClassifier(3, weight_type="no_weight", normalize=False)
#knn = KNNClassifier(3)
knn.fit(X_train, y_train)
score = knn.score(X_test, y_test)

In [467]:
score

array([0.80828083])

## Normalized

In [469]:
knn = KNNClassifier(3, weight_type="no_weight", normalize=True)
knn.fit(X_train, y_train)
score = knn.score(X_test, y_test)

In [470]:
score

array([0.83063306])

In [222]:
for i in range(1,16,2):
    knn = KNNClassifier(i, weight_type="no_weight")
    knn.fit(X_train, y_train)
    score = knn.score(X_test, y_test)
    print(i, score)

1 [0.81143114]
3 [0.83063306]
5 [0.84443444]
7 [0.84383438]
9 [0.84458446]
11 [0.84623462]
13 [0.84653465]
15 [0.84293429]


## Distance weighting

In [231]:
for i in range(1,16,2):
    knn = KNNClassifier(i, weight_type="inverse_distance")
    knn.fit(Xn_train, y_train)
    score = knn.score(Xn_test, y_test)
    print(i, score)

/Users/seong-euncho/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:55: RuntimeWarning: divide by zero encountered in power


1 [0.81143114]
3 [0.83108311]
5 [0.84563456]
7 [0.84908491]
9 [0.84848485]
11 [0.85283528]
13 [0.85163516]
15 [0.85073507]


# Housing

In [471]:
housing_train = arff.Arff(arff=r'housing_train.arff', label_count=1)
housing_test = arff.Arff(arff=r'housing_test.arff', label_count=1)
X_train = housing_train[:, :-1]
y_train = housing_train[:, -1].reshape(-1, 1)
X_test = housing_test[:, :-1]
y_test = housing_test[:, -1].reshape(-1, 1)
knn = KNNClassifier(3, output_type="continuous", weight_type="no_weight")
knn.fit(X_train, y_train)
y_hat = knn.predict(X_test)
score = knn.score(X_test, y_test)

In [472]:
score

16.598692810457514

In [300]:
for i in range(1,16,2):
    knn = KNNClassifier(i, output_type="continuou", weight_type="no_weight")
    knn.fit(X_train, y_train)
    score = knn.score(X_test, y_test)
    print(i, score)

1 24.60843137254902
3 16.598692810457514
5 15.896980392156863
7 18.969395758303325
9 20.658867102396517
11 23.498943445146654
13 24.51325443786983
15 24.278203921568636


## Distance weighting

In [301]:
for i in range(1,16,2):
    knn = KNNClassifier(i, output_type="continuous", weight_type="inverse_distance")
    knn.fit(X_train, y_train)
    score = knn.score(X_test, y_test)
    print(i, score)

1 24.60843137254902
3 16.361204005940902
5 12.119627591906251
7 10.736267400258754
9 11.345052919743303
11 11.551512562417233
13 11.655841849812564
15 11.880718647897593


# Credit Approval

In [486]:
len(input_type) - sum(input_type)

6

In [508]:
credit = arff.Arff(arff=r'credit.arff', label_count=1)
X = credit[:, :-1]
y = credit[:, -1].reshape(-1, 1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
input_type = np.array([1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0])
knn = KNNClassifier(13, output_type="nominal", input_type=input_type)
knn.fit(X_train, y_train)
knn.score(X_test, y_test)

array([0.8647343])

In [504]:
for i in range(1,16,2):
    knn = KNNClassifier(i, output_type="nominal", input_type=input_type)
    knn.fit(X_train, y_train)
    score = knn.score(X_test, y_test)
    print(i, score)

1 [0.79710145]
3 [0.85024155]
5 [0.83091787]
7 [0.83574879]
9 [0.84541063]
11 [0.84541063]
13 [0.85507246]
15 [0.84541063]


In [415]:
nom_diff = np.sum(X_nom != x_nom, axis=1)
cont_diff = X_cont - x_cont
cont_diff[np.isnan(cont_diff)] = 1

# Sklearn's implementation

In [510]:
from sklearn.neighbors import KNeighborsClassifier as KNC
from sklearn.neighbors import KNeighborsRegressor as KNR

## Telescope

### Raw

In [521]:
telescope_train = arff.Arff(arff=r'telescope_train.arff', label_count=1)
telescope_test = arff.Arff(arff=r'telescope_test.arff', label_count=1)
X_train = telescope_train[:, :-1]
y_train = telescope_train[:, -1]
X_test = telescope_test[:, :-1]
y_test = telescope_test[:, -1]
clf = KNC()
clf.fit(X_train, y_train)
clf.score(X_test, y_test)

0.8136813681368137

### Normalized

In [522]:
min_val = np.nanmin(X_train, axis=0)
max_val = np.nanmax(X_train, axis=0)
X_train = normalize(X_train)
X_test = normalize(X_test, min_val=min_val, max_val=max_val)
clf = KNC()
clf.fit(X_train, y_train)
clf.score(X_test, y_test)

0.8444344434443445

In [524]:
for i in range(1, 16, 2):
    clf = KNC(n_neighbors=i)
    clf.fit(X_train, y_train)
    score = clf.score(X_test, y_test)
    print(i, score)

1 0.8114311431143114
3 0.8306330633063307
5 0.8444344434443445
7 0.8438343834383438
9 0.8445844584458446
11 0.8462346234623462
13 0.8465346534653465
15 0.8429342934293429


### Inverse Distance

In [525]:
min_val = np.nanmin(X_train, axis=0)
max_val = np.nanmax(X_train, axis=0)
X_train = normalize(X_train)
X_test = normalize(X_test, min_val=min_val, max_val=max_val)
clf = KNC(weights="distance")
clf.fit(X_train, y_train)
clf.score(X_test, y_test)

0.8466846684668466

In [527]:
for i in range(1, 16, 2):
    clf = KNC(n_neighbors=i, weights="distance")
    clf.fit(X_train, y_train)
    score = clf.score(X_test, y_test)
    print(i, score)

1 0.8114311431143114
3 0.8325832583258326
5 0.8466846684668466
7 0.8478847884788479
9 0.8481848184818482
11 0.8505850585058505
13 0.8489348934893489
15 0.8474347434743474


### L1 norm

In [531]:
min_val = np.nanmin(X_train, axis=0)
max_val = np.nanmax(X_train, axis=0)
X_train = normalize(X_train)
X_test = normalize(X_test, min_val=min_val, max_val=max_val)
clf = KNC(n_neighbors=11, weights="distance", p=1)
clf.fit(X_train, y_train)
clf.score(X_test, y_test)

0.8517851785178517

## Housing

### Raw

In [533]:
housing_train = arff.Arff(arff=r'housing_train.arff', label_count=1)
housing_test = arff.Arff(arff=r'housing_test.arff', label_count=1)
X_train = housing_train[:, :-1]
y_train = housing_train[:, -1]
X_test = housing_test[:, :-1]
y_test = housing_test[:, -1]
regr = KNR()
regr.fit(X_train, y_train)
regr.score(X_test, y_test)

0.25491902354492035

### Normalized

In [534]:
min_val = np.nanmin(X_train, axis=0)
max_val = np.nanmax(X_train, axis=0)
X_train = normalize(X_train)
X_test = normalize(X_test, min_val=min_val, max_val=max_val)
regr = KNR()
regr.fit(X_train, y_train)
regr.score(X_test, y_test)

0.7925759551513474

In [535]:
for i in range(1, 16, 2):
    regr = KNR(n_neighbors=i)
    regr.fit(X_train, y_train)
    score = regr.score(X_test, y_test)
    print(i, score)

1 0.6789088086695408
3 0.7834200007163616
5 0.7925759551513474
7 0.7524870321621944
9 0.7304427840595424
11 0.6933854242239067
13 0.680150678778231
15 0.6832176215329073


### Inverse distance

In [536]:
for i in range(1, 16, 2):
    regr = KNR(n_neighbors=i, weights="distance")
    regr.fit(X_train, y_train)
    score = regr.score(X_test, y_test)
    print(i, score)

1 0.6789088086695411
3 0.7991753902998118
5 0.8411800448532063
7 0.8366218833290943
9 0.8173612829633798
11 0.7996109064914753
13 0.7897133604468035
15 0.7841246752040515


### L1 norm

In [537]:
regr = KNR(n_neighbors=5, weights="distance", p=1)
regr.fit(X_train, y_train)
regr.score(X_test, y_test)

0.8578979081717918

## Human Activity Recognition Using Smartphones Data Set

In [549]:
X_train = np.loadtxt("HAR_X_train.txt")
y_train = np.loadtxt("HAR_y_train.txt")
X_test = np.loadtxt("HAR_X_test.txt")
y_test = np.loadtxt("HAR_y_test.txt")

### Baseline

In [555]:
clf = KNC()
clf.fit(X_train, y_train)
clf.score(X_test, y_test)

0.9015948422124194

In [556]:
for i in range(1, 16, 2):
    clf = KNC(n_neighbors=i)
    clf.fit(X_train, y_train)
    score = clf.score(X_test, y_test)
    print(i, score)

1 0.8785205293518833
3 0.8907363420427553
5 0.9015948422124194
7 0.9032914828639295
9 0.9053274516457415
11 0.9046487953851374
13 0.9063454360366474
15 0.9043094672548354


In [560]:
for i in range(1, 16, 2):
    clf = KNC(n_neighbors=i, weights="distance")
    clf.fit(X_train, y_train)
    score = clf.score(X_test, y_test)
    print(i, score)

1 0.8785205293518833
3 0.8907363420427553
5 0.9002375296912114
7 0.9043094672548354
9 0.9060061079063454
11 0.9053274516457415
13 0.9066847641669494
15 0.9049881235154394


In [561]:
for i in range(5, 16, 2):
    clf = KNC(n_neighbors=i, weights="distance", p=1)
    clf.fit(X_train, y_train)
    score = clf.score(X_test, y_test)
    print(i, score)

5 0.9134713267729895
7 0.9127926705123854
9 0.9168646080760094
11 0.9212758737699356
13 0.9195792331184255
15 0.9178825924669155


### PCA

In [562]:
from sklearn.decomposition import PCA

In [563]:
pca = PCA(n_components=100)
pca.fit(X_train)

PCA(copy=True, iterated_power='auto', n_components=100, random_state=None,
    svd_solver='auto', tol=0.0, whiten=False)

In [574]:
for i in range(100):
    print(i, sum(pca.explained_variance_ratio_[:i]))

0 0
1 0.6255443998293534
2 0.6746746270487949
3 0.7158893015785994
4 0.7346388628001297
5 0.7515874626545213
6 0.7643081555473861
7 0.7760750069117569
8 0.7867647385552015
9 0.7964585363441172
10 0.805038718055507
11 0.8126617371636761
12 0.8193861937687167
13 0.8251803897253632
14 0.8307591961044213
15 0.8357484534049383
16 0.8404978296656398
17 0.8451698307587513
18 0.8494860093575678
19 0.8537431611559121
20 0.8578471115324018
21 0.8617871355416186
22 0.8655402286941136
23 0.8690645035649698
24 0.8724580977898989
25 0.8757794877585672
26 0.8789737756057521
27 0.8819915670727286
28 0.8849093927710497
29 0.8878050922775711
30 0.8906243701225776
31 0.8933914115129044
32 0.8960253619771138
33 0.8985784287191072
34 0.9009370874417739
35 0.9032436101601724
36 0.9054800914304529
37 0.9077095723823094
38 0.9098123319251135
39 0.9118962602285122
40 0.9139439973233491
41 0.9158725605365415
42 0.9177613552873035
43 0.9195731548268612
44 0.9213678789517435
45 0.9230911523383434
46 0.92469316595

In [577]:
pca = PCA(n_components=34)
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

In [583]:
for i in range(15, 32, 2):
    clf = KNC(n_neighbors=i, weights="distance", p=2)
    clf.fit(X_train_pca, y_train)
    score = clf.score(X_test_pca, y_test)
    print(i, score)

15 0.8914149983033594
17 0.8873430607397353
19 0.8859857482185273
21 0.8832711231761113
23 0.8853070919579233
25 0.8842891075670173
27 0.8849677638276213
29 0.8856464200882254
31 0.8839497794367153


## Greedy Wrapper

In [591]:
from sklearn.base import clone

In [632]:
def greedy_wrapper(model, X_train, X_test, y_train, y_test, max_features=40):
    num_features = X_train.shape[1]
    best_features = []
    for i in range(max_features+1):
        scores = np.zeros(num_features)
        for j in range(num_features):
            if j in best_features:
                continue
            wrap = best_features + [j]
            W_train = X_train[:, wrap]
            W_test = X_test[:, wrap]
            clf = clone(model)
            clf.fit(W_train, y_train)
            score = clf.score(W_test, y_test)
            scores[j] = score
        best = np.argmax(scores)
        best_features.append(best)
        print(best_features)
        print(np.max(scores))
        print()

In [633]:
clf = KNC(n_neighbors=11, weights="distance", p=1)
greedy_wrapper(clf, X_train, X_test, y_train, y_test)

[302]
0.47845266372582285

[302, 558]
0.7091957923311842

[302, 558, 166]
0.7828299966067187

[302, 558, 166, 52]
0.825924669155073

[302, 558, 166, 52, 228]
0.8598574821852731

[302, 558, 166, 52, 228, 120]
0.8788598574821853

[302, 558, 166, 52, 228, 120, 508]
0.8937902952154734

[302, 558, 166, 52, 228, 120, 508, 126]
0.9036308109942314

[302, 558, 166, 52, 228, 120, 508, 126, 128]
0.9131319986426875

[302, 558, 166, 52, 228, 120, 508, 126, 128, 95]
0.9209365456396336

[302, 558, 166, 52, 228, 120, 508, 126, 128, 95, 40]
0.9260264675941635

[302, 558, 166, 52, 228, 120, 508, 126, 128, 95, 40, 411]
0.9294197488971836

[302, 558, 166, 52, 228, 120, 508, 126, 128, 95, 40, 411, 464]
0.9304377332880895

[302, 558, 166, 52, 228, 120, 508, 126, 128, 95, 40, 411, 464, 481]
0.9321343739395996

[302, 558, 166, 52, 228, 120, 508, 126, 128, 95, 40, 411, 464, 481, 413]
0.9317950458092976

[302, 558, 166, 52, 228, 120, 508, 126, 128, 95, 40, 411, 464, 481, 413, 388]
0.9321343739395996

[302, 558,

KeyboardInterrupt: 

In [ ]:
clf = KNC(n_neighbors=11, weights="distance", p=2)
greedy_wrapper(clf, X_train, X_test, y_train, y_test)

[302]
0.47845266372582285

[302, 558]
0.7108924329826942

[302, 558, 69]
0.7807940278249067

[302, 558, 69, 559]
0.820834747200543

[302, 558, 69, 559, 456]
0.8466236851034951

[302, 558, 69, 559, 456, 434]
0.8696979979640312

[302, 558, 69, 559, 456, 434, 42]
0.8812351543942993

[302, 558, 69, 559, 456, 434, 42, 175]
0.8907363420427553

[302, 558, 69, 559, 456, 434, 42, 175, 201]
0.9032914828639295

[302, 558, 69, 559, 456, 434, 42, 175, 201, 451]
0.9110960298608755

[302, 558, 69, 559, 456, 434, 42, 175, 201, 451, 74]
0.9182219205972175

[302, 558, 69, 559, 456, 434, 42, 175, 201, 451, 74, 359]
0.9229725144214456

[302, 558, 69, 559, 456, 434, 42, 175, 201, 451, 74, 359, 445]
0.9253478113335596

[302, 558, 69, 559, 456, 434, 42, 175, 201, 451, 74, 359, 445, 184]
0.9321343739395996

[302, 558, 69, 559, 456, 434, 42, 175, 201, 451, 74, 359, 445, 184, 11]
0.9372242958941296

[302, 558, 69, 559, 456, 434, 42, 175, 201, 451, 74, 359, 445, 184, 11, 228]
0.9389209365456397

[302, 558, 69, 5

In [611]:
X_train[:, [9]].shape

(7352, 1)

In [610]:
X_train[:, 9][0]

-0.93472378